# 🪸 OpenClaw on Google Colab

Run your own **OpenClaw** AI gateway on a free Colab CPU instance — 3 clicks and you're live.

---
## Step 1 — Add Your API Key(s)

OpenClaw needs at least **one** LLM provider key. You set them up once in Colab's secure Secrets storage — they persist across sessions and never appear in the notebook.

### First time? Here's how:

1. Open the **🔑 Secrets** panel — click the key icon in the left sidebar
2. Click **“+ Add new secret”**
3. Type the **Name** exactly as shown below, paste your key as the **Value**
4. Toggle **“Notebook access”** ON

Add any of these (you only need one):

| Name (copy exactly) | Provider | Get a key | Free tier? |
|---|---|---|---|
| `GEMINI_API_KEY` | Google Gemini | [aistudio.google.com/apikey](https://aistudio.google.com/apikey) | ✅ Yes |
| `OPENROUTER_API_KEY` | OpenRouter | [openrouter.ai/keys](https://openrouter.ai/keys) | ✅ Some models |
| `ANTHROPIC_API_KEY` | Anthropic (Claude) | [console.anthropic.com](https://console.anthropic.com/settings/keys) | ❌ Pay-as-you-go |
| `OPENAI_API_KEY` | OpenAI (GPT) | [platform.openai.com/api-keys](https://platform.openai.com/api-keys) | ❌ Pay-as-you-go |

> 💡 **Easiest free start:** Get a Gemini key — it's free, takes 30 seconds, and works great.

### Already done this before?

Your secrets are saved in Colab. Just make sure **Notebook access** is toggled ON for this notebook, then move on to Step 2.

---
## Step 2 — Install & Configure

This cell does everything: installs Node.js + OpenClaw, detects your API keys, and lets you pick a model. Takes about 60 seconds.

In [ ]:
#@title ▶️ Click to install & configure
import subprocess, os, secrets, sys, time
from IPython.display import display, HTML

# ============================================================
# Phase 1: Install Node.js 22 + OpenClaw
# ============================================================
display(HTML('<b>⏳ Installing Node.js & OpenClaw...</b>'))

# Check Node.js
node_ok = False
try:
    ver = subprocess.run(['node', '-v'], capture_output=True, text=True)
    major = int(ver.stdout.strip().lstrip('v').split('.')[0])
    node_ok = major >= 22
except Exception:
    pass

if not node_ok:
    subprocess.run('curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash -',
                   shell=True, capture_output=True)
    subprocess.run(['sudo', 'apt-get', 'install', '-y', 'nodejs'], capture_output=True)

# Check OpenClaw
oc = subprocess.run(['which', 'openclaw'], capture_output=True, text=True)
if not oc.stdout.strip():
    r = subprocess.run(['npm', 'install', '-g', 'openclaw@latest'], capture_output=True, text=True)
    if r.returncode != 0:
        print(r.stderr[-1000:])
        raise RuntimeError('npm install failed')

node_v = subprocess.run(['node', '-v'], capture_output=True, text=True).stdout.strip()
oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()

display(HTML(f'''
<div style="background:#e8f5e9; border-radius:6px; padding:8px 14px; margin:4px 0; font-size:13px;">
  ✅ Node.js {node_v} &nbsp;·&nbsp; OpenClaw at <code>{oc_bin}</code>
</div>
'''))

# ============================================================
# Phase 2: Detect API keys from Colab Secrets
# ============================================================
try:
    from google.colab import userdata
    _has_colab = True
except ImportError:
    _has_colab = False

KEY_NAMES = ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GEMINI_API_KEY', 'OPENROUTER_API_KEY']

found = {}
for name in KEY_NAMES:
    val = None
    if _has_colab:
        try:
            val = userdata.get(name)
        except Exception:
            pass
    if not val:
        val = os.environ.get(name)
    if val and val.strip():
        found[name] = val.strip()
        os.environ[name] = val.strip()

if not found:
    display(HTML('''
    <div style="background:#fff3cd; border:1px solid #ffc107; border-radius:8px; padding:16px; margin:12px 0;">
      <b>⚠️ No API keys found.</b><br><br>
      Go back to <b>Step 1</b> and add at least one key in the 🔑 Secrets panel.<br>
      Make sure <b>"Notebook access"</b> is toggled ON, then re-run this cell.
    </div>
    '''))
else:
    # Show what we found
    provider_names = {
        'ANTHROPIC_API_KEY': 'Anthropic',
        'OPENAI_API_KEY': 'OpenAI',
        'GEMINI_API_KEY': 'Gemini',
        'OPENROUTER_API_KEY': 'OpenRouter',
    }
    detected = [provider_names[k] for k in found]

    display(HTML(f'''
    <div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:12px 16px; margin:8px 0;">
      ✅ <b>Keys detected:</b> {', '.join(detected)}
    </div>
    '''))

    # ============================================================
    # Phase 3: Model picker
    # ============================================================
    import ipywidgets as widgets

    MODELS = {}

    if 'ANTHROPIC_API_KEY' in found:
        MODELS['claude-opus-4-6   — Anthropic, strongest reasoning'] = 'claude-opus-4-6'
        MODELS['claude-sonnet-4-6 — Anthropic, fast & capable'] = 'claude-sonnet-4-6'

    if 'GEMINI_API_KEY' in found:
        MODELS['gemini-3-pro-preview   — Google, large context'] = 'gemini-3-pro-preview'
        MODELS['gemini-3-flash-preview — Google, fast & free-tier friendly'] = 'gemini-3-flash-preview'

    if 'OPENAI_API_KEY' in found:
        MODELS['gpt-5.2 — OpenAI, latest'] = 'gpt-5.2'

    if 'OPENROUTER_API_KEY' in found:
        MODELS['OpenRouter — auto, OpenClaw picks the best available'] = 'auto'

    labels = list(MODELS.keys())
    dropdown = widgets.Dropdown(
        options=labels,
        value=labels[0],
        description='Model:',
        style={'description_width': '55px'},
        layout=widgets.Layout(width='520px'),
    )

    def _on_change(change):
        os.environ['OPENCLAW_MODEL'] = MODELS[change['new']]

    dropdown.observe(_on_change, names='value')
    os.environ['OPENCLAW_MODEL'] = MODELS[labels[0]]

    display(HTML('<b>Pick a model:</b>'))
    display(dropdown)

    # ============================================================
    # Phase 4: Prepare state & .env
    # ============================================================
    STATE_DIR = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
    os.makedirs(STATE_DIR, exist_ok=True)
    os.environ['OPENCLAW_STATE_DIR'] = STATE_DIR

    token = secrets.token_hex(32)
    os.environ['OPENCLAW_GATEWAY_TOKEN'] = token

    with open(os.path.join(STATE_DIR, '.env'), 'w') as f:
        f.write(f'OPENCLAW_GATEWAY_TOKEN={token}\n')
        for k, v in found.items():
            f.write(f'{k}={v}\n')

    display(HTML(f'''
    <div style="background:#e3f2fd; border-radius:6px; padding:8px 14px; margin:8px 0; font-size:13px;">
      ℹ️ Select your model above, then run <b>Step 3</b> to start.
    </div>
    '''))

---
## Step 3 — Start OpenClaw

In [ ]:
#@title ▶️ Click to launch
import subprocess, os, time
from IPython.display import display, HTML

STATE = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
MODEL = os.environ.get('OPENCLAW_MODEL', 'claude-opus-4-6')

# Preflight checks
oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()
if not oc_bin:
    display(HTML('<div style="background:#f8d7da; padding:12px; border-radius:8px;">❌ OpenClaw not found. Run <b>Step 2</b> first.</div>'))
elif not os.environ.get('OPENCLAW_GATEWAY_TOKEN'):
    display(HTML('<div style="background:#f8d7da; padding:12px; border-radius:8px;">❌ No keys configured. Run <b>Step 2</b> first.</div>'))
else:
    # Environment
    env = os.environ.copy()
    env['OPENCLAW_STATE_DIR'] = STATE
    env['HOME'] = '/content'
    env['NODE_ENV'] = 'production'
    env['OPENCLAW_SKIP_CHANNELS'] = '1'

    # Symlink ~/.openclaw
    link = '/content/.openclaw'
    if os.path.islink(link):
        os.unlink(link)
    if not os.path.exists(link):
        os.symlink(STATE, link)

    # Stop previous instance
    subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
    time.sleep(2)

    # Launch
    log = open('/content/openclaw_gateway.log', 'w')
    proc = subprocess.Popen(
        [oc_bin, 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
        env=env, stdout=log, stderr=subprocess.STDOUT, preexec_fn=os.setsid,
    )

    display(HTML(f'<b>⏳ Starting OpenClaw with <code>{MODEL}</code>...</b>'))
    time.sleep(6)

    if proc.poll() is None:
        token_preview = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')[:16]
        display(HTML(f'''
        <div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:16px; margin:8px 0;">
          <h3 style="margin:0 0 10px 0">✅ OpenClaw is running!</h3>
          <table style="border:none; border-collapse:collapse; font-size:14px;">
            <tr><td style="padding:3px 12px 3px 0"><b>Model</b></td><td><code>{MODEL}</code></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Dashboard</b></td><td><code>http://localhost:18789</code></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Logs</b></td><td><code>/content/openclaw_gateway.log</code></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Token</b></td><td><code>{token_preview}...</code></td></tr>
          </table>
        </div>
        '''))
    else:
        with open('/content/openclaw_gateway.log') as f:
            err = f.read()[-3000:]
        display(HTML(f'''
        <div style="background:#f8d7da; border:1px solid #dc3545; border-radius:8px; padding:16px;">
          <b>❌ Gateway failed to start.</b>
          <pre style="margin-top:8px; font-size:12px; max-height:300px; overflow:auto;">{err}</pre>
        </div>
        '''))

---
## You're live! 🎉

The cells below are all **optional**. Use them if you need external access, a chat UI, or persistent storage.

---
### 🌐 Expose via ngrok

Access from your phone or connect messaging channels. Add `NGROK_AUTH_TOKEN` to Secrets (free at [ngrok.com](https://ngrok.com)).

In [ ]:
#@title ▶️ Open ngrok tunnel
import os
from IPython.display import display, HTML

tok = None
try:
    from google.colab import userdata
    tok = userdata.get('NGROK_AUTH_TOKEN')
except Exception:
    tok = os.environ.get('NGROK_AUTH_TOKEN')

if not tok:
    display(HTML('''
    <div style="background:#fff3cd; border:1px solid #ffc107; border-radius:8px; padding:14px;">
      <b>⚠️</b> Add <code>NGROK_AUTH_TOKEN</code> to the 🔑 Secrets panel, then re-run.
    </div>
    '''))
else:
    import subprocess
    subprocess.run(['pip', 'install', 'pyngrok', '-q'], check=True)
    from pyngrok import ngrok, conf
    conf.get_default().auth_token = tok.strip()
    tunnel = ngrok.connect(18789, 'http')
    display(HTML(f'''
    <div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:14px;">
      ✅ <b>Public URL:</b> <a href="{tunnel.public_url}" target="_blank">{tunnel.public_url}</a>
    </div>
    '''))

---
### 💬 Chat UI (Gradio)

In [ ]:
#@title ▶️ Launch chat
!pip install gradio requests -q

import gradio as gr, requests, os

URL = 'http://localhost:18789'
TOK = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')

def chat(message, history):
    try:
        r = requests.get(f'{URL}/health', timeout=5)
        if r.status_code != 200:
            return 'Gateway not responding. Run Step 3.'
    except Exception:
        return 'Gateway not running. Run Step 3.'
    try:
        r = requests.post(f'{URL}/api/chat',
            headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {TOK}'},
            json={'message': message, 'channel': 'webchat'}, timeout=120)
        if r.status_code == 200:
            d = r.json()
            return d.get('reply', d.get('message', str(d)))
        return f'Error {r.status_code}: {r.text[:500]}'
    except requests.exceptions.Timeout:
        return 'Request timed out.'
    except Exception as e:
        return f'Error: {e}'

gr.ChatInterface(fn=chat, type='messages', title='OpenClaw Chat',
    examples=['Hello! What can you do?', 'What model are you running?'],
).launch(share=False, debug=False)

---
### 💾 Google Drive Persistence

Save your state to Google Drive so it survives runtime recycles. **Requests Drive access.**

In [ ]:
#@title ▶️ Enable Drive storage
from google.colab import drive
import os, shutil
from IPython.display import display, HTML

drive.mount('/content/drive')

DRIVE = '/content/drive/MyDrive/openclaw/state'
LOCAL = '/content/openclaw_state'
os.makedirs(DRIVE, exist_ok=True)

if os.path.exists(LOCAL):
    for item in os.listdir(LOCAL):
        s, d = os.path.join(LOCAL, item), os.path.join(DRIVE, item)
        if os.path.isfile(s): shutil.copy2(s, d)
        elif os.path.isdir(s) and not os.path.exists(d): shutil.copytree(s, d)

os.environ['OPENCLAW_STATE_DIR'] = DRIVE
link = '/content/.openclaw'
if os.path.islink(link): os.unlink(link)
os.symlink(DRIVE, link)

display(HTML(f'''
<div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:14px;">
  ✅ <b>Drive persistence enabled.</b> Re-run Step 3 to restart with Drive storage.
</div>
'''))

---
### 🛠️ Utilities

In [ ]:
#@title 📜 View logs
!tail -50 /content/openclaw_gateway.log 2>/dev/null || echo 'No log file yet.'

In [ ]:
#@title 🟢 Check status
!pgrep -fa 'openclaw.*gateway' || echo '❌ Not running. Run Step 3.'

In [ ]:
#@title ⏹️ Stop gateway
!pkill -f 'openclaw.*gateway' && echo '✅ Stopped.' || echo 'Not running.'